In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
%matplotlib inline

import os, pickle, re, glob, time
import pandas as pd
import numpy as np
np.set_printoptions(precision=2)

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
from scipy import stats
from collections import Counter

sns.set_style('ticks')
pd.set_option('precision', 2)
#np.set_printoptions('precision', 2)

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

# Load data

In [2]:
args = pickle.load(open('arguments.pickle', 'rb'))
metadata = pd.read_csv(args.world_metadata_fname, sep='\t')

In [3]:
metadata = metadata[metadata['Host']=='Human']
metadata.shape

(2372808, 22)

In [4]:
metadata.columns

Index(['Virus name', 'Type', 'Accession ID', 'Collection date', 'Location',
       'Additional location information', 'Sequence length', 'Host',
       'Patient age', 'Gender', 'Clade', 'Pango lineage', 'Pangolin version',
       'Variant', 'AA Substitutions', 'Submission date', 'Is reference?',
       'Is complete?', 'Is high coverage?', 'Is low coverage?', 'N-Content',
       'GC-Content'],
      dtype='object')

In [5]:
metadata.loc[:, 'Collection date'] = pd.to_datetime(metadata['Collection date'])
metadata = metadata[metadata['Collection date'] >= '2020-01-01'].sort_values('Collection date')

In [6]:
metadata = metadata[['Accession ID','Location','Pango lineage','Clade','Collection date']]
metadata.head()

,Accession ID,Location,Pango lineage,Clade,Collection date
144124,EPI_ISL_579252,Oceania / New Zealand / Auckland,B.1.369,GH,2020-01-01
376265,EPI_ISL_831463,Europe / Germany / Berlin,B.1.177.86,GV,2020-01-01
376264,EPI_ISL_831462,Europe / Germany / Berlin,B.1.389,GH,2020-01-01
376263,EPI_ISL_831461,Europe / Germany,B.1,G,2020-01-01
376262,EPI_ISL_831460,Europe / Germany / Berlin,B.1.221,G,2020-01-01


In [7]:
metadata.loc[:, 'Collection date'] = pd.to_datetime(metadata['Collection date'])

In [8]:
metadata['Accession ID'] = [i.split('_')[2] for i in metadata['Accession ID']]

In [9]:
metadata.loc[:, 'Year'] = metadata['Collection date'].dt.strftime('%Y')
metadata.loc[:, 'Month'] = metadata['Collection date'].dt.strftime('%m')
metadata.loc[:, 'Year-Month'] = metadata['Collection date'].dt.strftime('%Y-%m')

In [10]:
metadata['region'] = [(i.split('/')[0]).strip() for i in metadata['Location']]
metadata['country'] = [(i.split('/')[1]).strip() for i in metadata['Location']]

In [11]:
month_list = sorted(list(set(metadata['Year-Month'])))
month_list

['2020-01',
 '2020-02',
 '2020-03',
 '2020-04',
 '2020-05',
 '2020-06',
 '2020-07',
 '2020-08',
 '2020-09',
 '2020-10',
 '2020-11',
 '2020-12',
 '2021-01',
 '2021-02',
 '2021-03',
 '2021-04',
 '2021-05',
 '2021-06',
 '2021-07']

In [12]:
for i,m in enumerate(month_list):
    temp_df = metadata[metadata['Year-Month']==m]
    temp_df = temp_df.groupby(['country','Pango lineage']).count().sort_values(by='country') 

    temp_df = temp_df[['Collection date']]
    temp_df.columns = ['count']

    temp_df['country_1']=[x[0] for x in temp_df.index]
    temp_df['lin']=[x[1] for x in temp_df.index]
    
    sum_country = temp_df.groupby(['country_1']).sum()
    sum_country_dict = sum_country.to_dict()['count']

    temp_df['sum'] = [sum_country_dict[i] for i in list(temp_df['country_1'])]
    temp_df['percentage'] = temp_df['count']/temp_df['sum']*100    
    temp_df['month'] = str(m)
    
    if i==0:
        count_metadata_df = temp_df
    else:
        count_metadata_df = pd.concat([count_metadata_df,temp_df])

In [13]:
count_metadata_df

count       country_1        lin    sum  \
country        Pango lineage                                            
Argentina      None               1       Argentina       None      1   
Armenia        B.1.1              1         Armenia      B.1.1      3   
               B.4                2         Armenia        B.4      3   
Australia      None               2       Australia       None    453   
               D.3                6       Australia        D.3    453   
...                             ...             ...        ...    ...   
United Kingdom B.1.1.7          108  United Kingdom    B.1.1.7  19804   
               B.1.526            1  United Kingdom    B.1.526  19804   
               B.1.617.1          1  United Kingdom  B.1.617.1  19804   
               B.1.617.2      19681  United Kingdom  B.1.617.2  19804   
               P.1                2  United Kingdom        P.1  19804   

                              percentage    month  
country        Pango lineage                       
Argentina      None             1.00e+02  2020-01  
Armenia        B.1.1            3.33e+01  2020-01  
               B.4              6.67e+01  2020-01  
Australia      None             4.42e-01  2020-01  
               D.3              1.32e+00  2020-01  
...                                  ...      ...  
United Kingdom B.1.1.7          5.45e-01  2021-07  
               B.1.526          5.05e-03  2021-07  
               B.1.617.1        5.05e-03  2021-07  
               B.1.617.2        9.94e+01  2021-07  
               P.1              1.01e-02  2021-07  

[34495 rows x 6 columns]

In [16]:
count_metadata_df.to_csv(os.path.join(args.preprocess_dir, 'count_metadata_df.csv'), index=False)

In [14]:
# count_metadata_df[(count_metadata_df['lin']=='B.1.617.2')].sort_values('country')

In [15]:
# count_metadata_df[(count_metadata_df['country_1']=='Thailand')]